# 04 - Applied ML

In [1]:
# import all the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

## Cleaning the data

In [41]:
# import the data
df = pd.read_csv('CrowdstormingDataJuly1st.csv')

# initial shape of the data
print(df.shape)

(146028, 28)


In [42]:
# check which columns have NaNs
df.isnull().any()

playerShort      False
player           False
club             False
leagueCountry    False
birthday         False
height            True
weight            True
position          True
games            False
victories        False
ties             False
defeats          False
goals            False
yellowCards      False
yellowReds       False
redCards         False
photoID           True
rater1            True
rater2            True
refNum           False
refCountry       False
Alpha_3           True
meanIAT           True
nIAT              True
seIAT             True
meanExp           True
nExp              True
seExp             True
dtype: bool

We can drop the column photoID since we won't use the photos of the players.

In [43]:
df = df.drop(['photoID'], axis=1)

We can also remove the column Alpha_3 since the same information seems to be represented by the column refCountry.

In [44]:
df = df.drop(['Alpha_3'], axis=1)

In [45]:
# remove records that have rater1 and/or rater2 as NaNs
df = df.dropna(axis=0, subset=['rater1', 'rater2'])

# create a new column with the mean of the two raters
df['raterMean'] = (df['rater1'] + df['rater2']) / 2

In [46]:
df.shape

(124621, 27)

Let's now focus on the referees. As stated in [this notebook](http://nbviewer.jupyter.org/github/mathewzilla/redcard/blob/master/Crowdstorming_visualisation.ipynb), we should only take into account referees that took part in games of one of the considered leagues. This means that referees with less than 22 dyads (corresponding to the, at least, 22 players of a game) shouldn't be relevant.

In [47]:
refs = df['refNum'].value_counts()

df = df[df['refNum'].isin(refs[refs > 21].index)]

In [48]:
df.shape

(112484, 27)

In [49]:
#TODO: find a better way to clean the data
# drop records with NaNs
df.dropna(axis=0,inplace=True)

In [54]:
#TODO: group results by player + performe the following opération : remove id, name etc...
df.shape

(104420, 27)

In [52]:
# deals with labels : dummy codes .... need to clean a bit the df :D
pd.get_dummies(df, prefix=['dummyCode1_','dummyCode2_','dummyCode3_','dummyCode4_','dummyCode5_','dummyCode6_']).head()

,height,weight,games,victories,ties,defeats,goals,yellowCards,yellowReds,redCards,...,dummyCode6__Center Forward,dummyCode6__Center Midfielder,dummyCode6__Defensive Midfielder,dummyCode6__Goalkeeper,dummyCode6__Left Fullback,dummyCode6__Left Midfielder,dummyCode6__Left Winger,dummyCode6__Right Fullback,dummyCode6__Right Midfielder,dummyCode6__Right Winger
5,182.0,71.0,1,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,187.0,80.0,1,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,180.0,68.0,1,0,0,1,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,193.0,80.0,1,0,1,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9,180.0,70.0,1,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


## Training the Classifier

In [151]:
X_train = df[['height', 'weight', 'games','victories','defeats','goals','meanIAT','nIAT','seIAT','meanExp','nExp','seExp']]
Y_train = np.asarray(df['raterMean'], dtype="|S6")

In [152]:
clf = RandomForestClassifier(n_estimators=50)

In [153]:
clf = clf.fit(X_train, Y_train)

In [154]:
# error on the training set
train_error = 1.0 - clf.score(X_train, Y_train)
print('Error on the training set:', train_error)

Error on the training set: 0.123778969546
